In [ ]:
import pandas
import numpy
from matplotlib import pyplot as plt
import seaborn
%matplotlib notebook
import datetime
import general
import discretionary_aua
import vantage_aua
import data_accessing
import combined
import revenue
import costs
import consolidated
import discf
from ipywidgets import widgets
from IPython.display import display
import stats
pandas.set_option('display.float_format', '{:20,.4f}'.format)   #suppressing scientific notation
idx = pandas.IndexSlice

In [ ]:
print('Pulling data...')
fund_dic = data_accessing.read_data(data_accessing.fund_data_name,data_accessing.fund_data_types)
nnb_dic = data_accessing.read_data(data_accessing.nnb_data_name,data_accessing.nnb_data_sheet)
discretionary_aua.append_share_class_units(fund_dic) # appending share class units
index_hl_dic = data_accessing.read_data(data_accessing.index_hl_data_name, data_accessing.index_hl_data_sheet)
data_dic = {**fund_dic, **index_hl_dic, **nnb_dic}

input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)

print('Data pulled successfully.')

In [ ]:
assumption_button = widgets.Button(description='Update assumptions',layout=widgets.Layout(width='20%',height='40px'))
assumption_button.style.button_color = 'lightgreen'
def update_assumption(b):
    global input_dic
    input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)
    print ('Assumptions updated successfully...')

assumption_button.on_click(update_assumption)
display(assumption_button)

In [ ]:
revenue_types = ['Semi-annual revenue','Annual revenue - FY','Annual revenue - CY','Consolidated semi-annual revenue','Consolidated annual revenue - FY','Consolidated annual revenue - CY']
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch',border='solid',width='30%')
items = {}
for w in revenue_types:
    items[w] = widgets.Button(description=w,layout=widgets.Layout(width='100%', height='40px'))
    items[w].style.button_color = 'lightgreen'
left_box = widgets.VBox([list(items.values())[0],list(items.values())[1],list(items.values())[2]],layout=box_layout)
right_box = widgets.VBox([list(items.values())[3],list(items.values())[4],list(items.values())[5]],layout=box_layout)
items['Semi-annual revenue'].on_click(lambda x: display(revenue.semi_revenue(data_dic,input_dic)))
items['Annual revenue - FY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic)))
items['Annual revenue - CY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic,cal_year=True)))
items['Consolidated semi-annual revenue'].on_click(lambda x: display(consolidated.revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - FY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - CY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic,cal_year=True)))
widgets.HBox([left_box,right_box])



In [ ]:
costs_types = ['Semi-annual costs','Annual costs - FY','Annual costs - CY','Consolidated semi-annual costs','Consolidated annual costs - FY','Consolidated annual costs - CY']
items2 = {}
for w2 in costs_types:
    items2[w2] = widgets.Button(description=w2,layout=widgets.Layout(width='100%', height='40px'))
    items2[w2].style.button_color = 'lightgreen'
left_box2 = widgets.VBox([list(items2.values())[0],list(items2.values())[1],list(items2.values())[2]],layout=box_layout)
right_box2 = widgets.VBox([list(items2.values())[3],list(items2.values())[4],list(items2.values())[5]],layout=box_layout)
items2['Semi-annual costs'].on_click(lambda x: display(costs.semi_costs(input_dic)))
items2['Annual costs - FY'].on_click(lambda x: display(costs.annual_costs(input_dic)))
items2['Annual costs - CY'].on_click(lambda x: display(costs.annual_costs(input_dic,cal_year=True)))
items2['Consolidated semi-annual costs'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - FY'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - CY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic,cal_year=True)))
widgets.HBox([left_box2,right_box2])

In [ ]:
ops = [i for i in list(range(general.recent_end_year, general.recent_end_year+10))]
a_slider = widgets.SelectMultiple(options=ops, description='Years for comparison')
display(a_slider)

In [ ]:
compare1 = ['NNB comparison','AUA comparison']
box_layout2 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='30%')
items3 = {}
for w3 in compare1:
    items3[w3] = widgets.Button(description=w3,layout=widgets.Layout(width='100%', height='40px'))
    items3[w3].style.button_color = 'lightgreen'
box3 = widgets.Box([list(items3.values())[0],list(items3.values())[1]],layout=box_layout2)
items3['NNB comparison'].on_click(lambda x: display(consolidated.get_nnb_compare(data_dic, input_dic,list(a_slider.value))))
items3['AUA comparison'].on_click(lambda x: display(consolidated.get_aua_compare(data_dic, input_dic,list(a_slider.value))))


display(box3)

In [ ]:
compare2 = ['Interim revenue comparison','Annual revenue comparison', 'Interim costs comparison', 'Annual costs comparison']
box_layout3 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='50%')
items4 = {}
for w4 in compare2:
    items4[w4] = widgets.Button(description=w4,layout=widgets.Layout(width='100%', height='40px'))
    items4[w4].style.button_color = 'lightgreen'
top_box4 = widgets.HBox([list(items4.values())[0],list(items4.values())[1]],layout=box_layout3)
bottom_box4 = widgets.HBox([list(items4.values())[2],list(items4.values())[3]],layout=box_layout3)
items4['Interim revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,True,list(a_slider.value))))
items4['Annual revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,False,list(a_slider.value))))
items4['Interim costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,True,list(a_slider.value))))
items4['Annual costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,False,list(a_slider.value))))

widgets.VBox([top_box4, bottom_box4])

In [ ]:
'''=====current values=======
discounted cash flow period: 9 years
perpetuity growth rate: 0.035
cash flow discounted rate: 0.085
'''
dcf_period = widgets.IntText(description='DCF period - No. of years:')
display(dcf_period)
perp_growth = widgets.FloatText(description='Perpetuity growth rate:')
display(perp_growth)
disct_rate = widgets.FloatText(description='Cash flow discount rate:')
display(disct_rate)
current_value = widgets.Checkbox(value=False, description='Fair value now')
display(current_value)

In [ ]:
dis_cash = ['Discounted cash flow','Fair value']
items5 = {}
for w5 in dis_cash:
    items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'))
    items5[w5].style.button_color = 'lightgreen'
fair = widgets.HBox([list(items5.values())[0],list(items5.values())[1]],layout=box_layout3)
items5['Discounted cash flow'].on_click(lambda x: display(discf.disc_cash_flow(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value)))
items5['Fair value'].on_click(lambda x: display(discf.fair_value(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value,pep_rate=perp_growth.value)))    
display(fair)

In [ ]:
others = ['HLF composite monthly return','Total NNB','NNB distribution','Total AUA','% Revenue distribution','Avg AUA distribution']
items6 = {}
for w6 in others:
    items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'))
    items6[w6].style.button_color = 'lightgreen'
other_box1 = widgets.VBox([list(items6.values())[0],list(items6.values())[1],list(items6.values())[2]],layout=box_layout)
other_box2 = widgets.VBox([list(items6.values())[3],list(items6.values())[4],list(items6.values())[5]],layout=box_layout)
items6['HLF composite monthly return'].on_click(lambda x: display(discretionary_aua.get_composite_return(data_dic)))
items6['Total NNB'].on_click(lambda x: display(combined.total_nnb(data_dic, input_dic)))
items6['NNB distribution'].on_click(lambda x: display(combined.nnb_distribution(data_dic, input_dic)))
items6['Total AUA'].on_click(lambda x: display(combined.total_aua(data_dic, input_dic)))
items6['% Revenue distribution'].on_click(lambda x: display(stats.summary_revenue_dist_percent(data_dic, input_dic)))
items6['Avg AUA distribution'].on_click(lambda x: display(stats.summary_avg_aua_dist(data_dic, input_dic)))


widgets.HBox([other_box1, other_box2])

#Financial year based

In [ ]:
nnb_radio = widgets.RadioButtons(options=['day','month','quarter','annual'],description='Period',disabled=False)
nnb_dropdown = widgets.Dropdown(options=['no_select',None],description='Fund option',disabled=False)
display(nnb_radio)
display(nnb_dropdown)

nnb_b = widgets.Button(description='Implied HLF nnb period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b.style.button_color = 'lightgreen'
nnb_b.on_click(lambda x: display(stats.hlf_to_date_implied_nnb(data_dic,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))
display(nnb_b)

In [ ]:
cash_radio = widgets.RadioButtons(options=['monthly','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(cash_radio)


cash_b = widgets.Button(description='Cash margin', layout=widgets.Layout(width='50%', height='40px'))
cash_b.style.button_color = 'lightgreen'
cash_b.on_click(lambda x: display(stats.cash_margin(data_dic,cash_radio.value)))
display(cash_b)

In [ ]:
eps_radio = widgets.RadioButtons(options=[True, False],description='Calendar year?',disabled=False)
display(eps_radio)

eps_summary = widgets.Button(description='Net earnings and EPS', layout=widgets.Layout(width='50%', height='40px'))
eps_summary.style.button_color = 'lightgreen'
eps_summary.on_click(lambda x: display(stats.summary_total(data_dic, input_dic, cal=eps_radio.value)))
display(eps_summary)


In [ ]:
# %load_ext autoreload
# %autoreload 2